# Creating custom MCP tools in database

MCP **Tools for databases are just end points** to access the data products available there.

We have two main scenarios for analytic tools:
* Unknown questions --> Generic "database" tools --> Exploratory analysis
  * *list table, sample table, run SQL query...*
* Known questions --> Specific **"business" tools** --> Run day-to-day business
  * *break down revenue by segments and region, forecast sales, get customer profile...*

Very often, **business tools** are nothing but prepared queries and associated semantics against a specific data product for a specific consumer.

So a **MCP tool for a specific business purpose should just be the last processing step in a data pipeline**, and building such tool should naturally be a data engineering task... 

So wouldn't it be more natural to create MCP tools in-database, to package it alongside the data pipelines and deploy it together, rather than with MCP server configuration?

We will demonstrate how you can create MCP tools for your specific business purposes without touching MCP server code or configuration, or even restarting it!

In [185]:
%connect local, hidewarnings=True

Connected: 'local' connection activated for user 'demo_user'


## Pre-requisite

You need to create the MCP tool registry database and structures in your database. 
Use the code is in the [registry_setup.sql](examples/server-customisation/registry_setup.sql) file.

The code below assumes that you have deployed the registry structures in the database `mcp`

## A season's "hello world":

Many database objects can embed business logic, but UDFs, Macros and Views are particularly convenient for this task.

SQL UDFs are the simplest form, but the same principle apply to other language UDFs and STOs...

In [168]:
REPLACE FUNCTION demo_user.season_greetings (user_name varchar(100))
RETURNS VARCHAR(200)
LANGUAGE SQL
CONTAINS SQL
DETERMINISTIC
SQL SECURITY DEFINER
COLLATION INVOKER
INLINE TYPE 1
RETURN 'HO HO HO Merry Christmas '||trim(user_name)||'!';

Success: 5 rows affected

The nice thing about these objects is that we can directly document it in database:

In [169]:
comment on demo_user.season_greetings as 'Turns Christmas spirit on for better results. Use before every database interaction in December. Very important.'

Success: 1 rows affected

In [170]:
comment on demo_user.season_greetings.user_name as 'Name of the invoker of the tool (Agent or end user).'

Success: 1 rows affected

In [171]:
comment on demo_user.season_greetings.return0 as 'A joyful greeting message.'

Success: 1 rows affected

Now let's "register" our tool in a special database

In [172]:
ins mcp.mcp_tool(toolname, databasename, tablename) values('season_greetings', 'demo_user', 'season_greetings');
ins mcp.mcp_tool_user_access sel 'season_greetings', user;
ins mcp.mcp_tool_tag sel 'season_greetings', 'december-special';

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

In [173]:
sel * from mcp.mcp_list_tools;
sel * from mcp.mcp_list_toolparams

,ToolName,DataBaseName,TableName,toolType,registered_ts,docstring,Tags
1,season_greetings,demo_user,season_greetings,F,2025-12-17 20:01:35.620000,Turns Christmas spirit on for better results. Use before every database interaction in December. Very important. Arguments: user_name - Name of the invoker of the tool (Agent or end user).,december-special


,ToolName,ParamName,ParamType,ParamLength,ParamPosition,ParamRequired,ParamComment
1,season_greetings,user_name,CV,100,1,Y,Name of the invoker of the tool (Agent or end user).


Now connect to your MCP server and try this tool. If the server is already running, you need to re-connect the server with your prefered client to trigger the loading of the new tools.

## Something more useful

### Create and document tools for banking churn prediction

Run the [Banking Custmer churn Notebook](UseCases/Banking_Customer_Churn/Banking_Customer_Churn_Python.ipynb) to load data and a ML Model. 
Get the final model scorign query with `print(result_xgb.show_query())`

That gives us a:
- "source" dataset with customer profile information
- a mechanism to score their churn risk using BYOM

In [17]:
sel top 5 * from DEMO_BankChurn.customer_churn

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,3487,15630661,Vasilyev,614,Spain,Female,25,10,75212.28,1,1,0,58965.04,0
2,1407,15629617,Cook,572,Spain,Male,23,2,126873.52,1,0,1,67040.12,0
3,3956,15695168,Bruce,625,France,Male,39,2,.00,2,1,0,100403.05,0
4,6036,15764021,Frolov,617,France,Male,34,1,61687.33,2,1,0,105965.25,0
5,8585,15732967,Cremonesi,731,France,Male,19,6,.00,2,1,1,151581.79,0


In [26]:
SELECT top 5 * FROM "mldb".PMMLPredict(
	ON (select "CustomerId","HasCrCard","IsActiveMember","Exited","Gender","France","Germany","Spain","OneProduct","TwoProduct","ThreeProduct","FourProduct","Age","Balance","CreditScore","EstimatedSalary","Tenure" from (select * from "DEMO_USER"."ml__sample__1765985670504639" where "sampleid" = 2) as temp_table) AS InputTable
	PARTITION BY ANY 
	ON (select model_id,model from "DEMO_USER"."bank_models" where model_id = 'xgb') AS ModelTable
	DIMENSION
	USING
	Accumulate('CustomerId', 'Exited')
	ModelOutputFields('probability(1)','probability(0)')
) as sqlmr

,CustomerId,Exited,prediction,probability(1),probability(0)
1,15688963,1,,0.8192829362241605,0.18071706377583951
2,15674811,0,,0.01468511180013946,0.9853148881998606
3,15706602,0,,0.018744890949974554,0.9812551090500254
4,15809826,1,,0.5238608217183465,0.47613917828165353
5,15603378,0,,0.035517400540522076,0.9644825994594779


Let's make tools out of these queries:
- Give us the top customers at risk per geography
- Give us the customer profile information based on CustomerId

...additionally trace why specific customer profiles are being accessed.

In [174]:
REPLACE MACRO get_CustomersAtRisk(
    Geography VARCHAR(128)) 
AS (
    SELECT top 5 CustomerId, "probability(1)" FROM "mldb".PMMLPredict(
        ON (
            select "CustomerId","HasCrCard","IsActiveMember","Exited","Gender","France","Germany","Spain","OneProduct","TwoProduct","ThreeProduct","FourProduct","Age","Balance","CreditScore","EstimatedSalary","Tenure" 
            from (
                select p.* 
                from "DEMO_USER"."ml__sample__1765985670504639" p
                join DEMO_BankChurn.customer_churn s
                    on s.CustomerId=p.CustomerId
                    and (COALESCE(:Geography, '') = '' OR Geography = :Geography)
                where "sampleid" = 2
        ) as temp_table) AS InputTable
        PARTITION BY ANY 
        ON (select model_id,model from "DEMO_USER"."bank_models" where model_id = 'xgb') AS ModelTable
        DIMENSION
        USING
        Accumulate('CustomerId')
        ModelOutputFields('probability(1)')
    ) as sqlmr
    order by 2 desc;
);

Success: 2 rows affected

In [175]:
comment on demo_user.get_CustomersAtRisk as 'Returns the top 5 customers with the highest churn risk at the moment for a given country';

Success: 1 rows affected

In [176]:
comment on demo_user.get_CustomersAtRisk.Geography as 'Country name in English';

Success: 1 rows affected

In [177]:
exec get_CustomersAtRisk('spain')

,CustomerId,probability(1)
1,15731815,0.9991789996149415
2,15654562,0.997877950621257
3,15623107,0.9906570190012446
4,15589475,0.9900626698420565
5,15647898,0.988911924832929


In [64]:
create multiset table customer_inquiries (customerId int, reason varchar(1000), inquired_ts timestamp)

Success: 0 rows affected

Macros also give us the ability to specifiy an entire workflow, with multiple read/write operations in database...

In [178]:
REPLACE MACRO get_CustomersProfile(
    customerId INT NOT NULL,
    reason varchar(1000) NOT NULL) 
AS (
    SELECT * from DEMO_BankChurn.customer_churn s where s.CustomerId=:CustomerId;
    
    ins customer_inquiries(:CustomerId, :reason, current_timestamp);
);

Success: 3 rows affected

In [179]:
comment on demo_user.get_CustomersProfile as 'Returns customer profile information given a customer ID. You must specify the reason for inquiry.';

Success: 1 rows affected

In [180]:
comment on demo_user.get_CustomersProfile.customerId as 'Customer ID';

Success: 1 rows affected

In [181]:
comment on demo_user.get_CustomersProfile.reason as 'Reason for inquiry, in English.';

Success: 1 rows affected

In [182]:
EXEC get_CustomersProfile(15623107,'just browsing... ;)')

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,4350,15623107,Chukwumaobim,686,Spain,Male,45,3,74274.87,3,1,0,64907.48,1


Success: 1 rows affected

In [183]:
sel customer_inquiries.*

,customerId,reason,inquired_ts
1,15623107,just browsing... ;),2025-12-17 20:07:31.190000


### Register tools

In [184]:
ins mcp.mcp_tool(toolname, databasename, tablename) values('get_CustomersAtRisk', 'demo_user', 'get_CustomersAtRisk');
ins mcp.mcp_tool_user_access sel 'get_CustomersAtRisk', user;
ins mcp.mcp_tool_tag sel 'get_CustomersAtRisk', 'churn';

ins mcp.mcp_tool(toolname, databasename, tablename) values('get_CustomersProfile', 'demo_user', 'get_CustomersProfile');
ins mcp.mcp_tool_user_access sel 'get_CustomersProfile', user;
ins mcp.mcp_tool_tag sel 'get_CustomersProfile', 'churn';

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Success: 1 rows affected

Now connect to your MCP server and try this tool. If the server is already running, you need to re-connect the server with your prefered client to trigger the loading of the new tools.

## Reset this demo

In [186]:
del from mcp.mcp_tool;
del from mcp.mcp_tool_user_access;
del from mcp.mcp_tool_tag;

Success: 3 rows affected

Success: 3 rows affected

Success: 3 rows affected

In [187]:
del from customer_inquiries;

Success: 2 rows affected